#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [129]:
# names here
# Alan Tran 
# John R. Smith

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [130]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [131]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features # a list of feature values for this data point

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [132]:
def get_data(filename):
    data = []
    df = pd.read_csv(filename, header=None)
    
    for index, row in df.iterrows() : 
        dp = DataPoint(label=row[19], features=row[0:19]) # 19 is the class label, whether someone has stuff
        data.append(dp)
    return data
silly_data = get_data('subset.txt')

In [133]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [134]:
def make_prediction(tree_root, data_point):
#     your code goes here

    return None

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [135]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
    for dp in data:
        if dp.features[feature_idx] <= threshold :
            left_split.append(dp)
        else :
            right_split.append(dp)
#     your code goes here
    return (left_split, right_split)

test_l, test_r = split_dataset(silly_data, 1, 0.5)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [145]:
def calc_entropy(data):
    entropy = 0.0
    yes = 0
    no = 0
    for dp in data :
        if (dp.label == 1) :
            yes = yes + 1
        else : 
            no = no + 1
    sum = yes + no

    # print("num yes: ", yes)
    # print("num no: ", no)
    # print("num sum: ", sum)
    if yes == 0 or no == 0 :
        return 1
    else : 
        return -(yes/sum) * log2(yes/sum) - (no/sum) * log2(no/sum)



Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [148]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
    parent_entropy = calc_entropy(data)
    ft_values = []

    # sort data by feature_idx
    data = sorted(data, key=lambda x: x.features[feature_idx])
    index = 1

    for index in range(len(data)) :
        if index == 0 : 
            continue
        # Check if right and left node feature values differ
        if (data[index-1].label != data[index].label) : 
            cur_thresh = (data[index-1].features[feature_idx] + data[index].features[feature_idx])/2
            left, right = split_dataset(data, feature_idx, cur_thresh)
            e_left = calc_entropy(left)
            e_right = calc_entropy(right)
            cur_entropy = ((len(left)/len(data)) * e_left) + ((len(right)/len(data)) * e_right)
            cur_info_gain = parent_entropy - cur_entropy
            # print(cur_info_gain)
            # print(cur_thresh)
            if cur_info_gain > best_info_gain : 
                best_info_gain = cur_info_gain
                best_thresh = cur_thresh
    
    return (best_info_gain, best_thresh)

for i in range(2, 18) :
    test_info_gain, test_threshold = calc_best_threshold(silly_data, i)
    print(i, "   ", test_info_gain)
    print(i, "   ", test_threshold)

2     0.256425891682003
2     26.5
3     0.256425891682003
3     26.5
4     0.256425891682003
4     25.5
5     0.256425891682003
5     23.0
6     0.256425891682003
6     21.5
7     0.256425891682003
7     15.0
8     0.12451124978365313
8     34.411814
9     0.12451124978365313
9     13.915189000000002
10     0.12451124978365313
10     2.815597
11     0.0912774462416801
11     0.6340665
12     0.12451124978365313
12     0.0
13     0.256425891682003
13     0.0
14     0.256425891682003
14     0.0
15     0.256425891682003
15     0.0
16     0.0912774462416801
16     0.493488
17     0.256425891682003
17     0.11579149999999999


Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [138]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [139]:
def create_leaf_node(data):
#     your code goes here
    return None

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root). [Hint: this is where the recursion happens.]

In [140]:
def create_decision_tree(data, max_levels):
#     your code goes here
    return None

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [141]:
def calc_accuracy(tree_root, data):
#     your code goes here
    return 0.0

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [142]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

# partition data into train_set and test_set
train_set = None
test_set = None

print ('Training set size:', len(train_set))
print ('Test set size    :', len(test_set))

# the timer is just for fun! you will NOT be graded on runtime
start = time.time()

# create the decision tree
tree = create_decision_tree(train_set, 10)

end = time.time()
print ('Time taken:', end - start)

# calculate the accuracy of the tree
accuracy = calc_accuracy(tree, test_set)
print ('The accuracy on the test set is ', str(accuracy * 100.0))
#t.printTree()

TypeError: object of type 'NoneType' has no len()